In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json
import pandas


In [125]:
records = []
for line in open('upload_600ms.log'):
    # evt = AciEventUpdate(1) SensorValues: sensor_id:15 proximity_ids:[12, 20, 24, 0, 0] proximity_rssi:[53, 55, 57, 0, 0] battery:194, accel:(37, 0, 144), status:0, valid_time:1501623900
    m = re.search('^evt = AciEventUpdate.*SensorValues.*sensor_id:(\d+) .*valid_time:(\d+)$', line)
    

    if m:
        epoch_timestamp = int(m.group(2))
        if epoch_timestamp < 200000000:
            continue
        timestamp = datetime.utcfromtimestamp(epoch_timestamp).isoformat()
        sensor_col = '%02d' % int(m.group(1))
        records.append({'ts':timestamp, sensor_col:1})
        
df = pandas.DataFrame(records)
df.set_index(['ts'])

# group by timestamp, and skip first 5 rows, as sensors are syncing up
df = df.groupby("ts").sum().iloc[4:]
df.reindex_axis(sorted(df.columns), axis=1)
pandas.set_option('display.max_columns', None)
df.tail(10) # Show layout of dataframe

,02,03,07,08,11,12,15,18,20,24,27,39,40,41,42,43,44,45,46,47,48,49,50,51,55,61
ts,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-11T18:27:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:27:10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:27:20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:27:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:27:40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN
2017-08-11T18:27:50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:28:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:28:10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T18:28:20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


In [126]:
df.shape

(63, 26)

In [127]:
# Overall participation
counts = df.sum().dropna()
overall = counts.sum() / (counts.shape[0] * df.shape[0]) * 100
print "%0.2f%% overall participation" % overall

# Individual participation
sensor_participation = counts / df.shape[0] * 100
print "-------------------------------"
print "individual sensor participation"
for (sensor_id, pct) in sensor_participation.iteritems():
    print "%2d = %6.2f%%" % (int(sensor_id), pct)

91.58% overall participation
-------------------------------
individual sensor participation
 2 =  95.24%
 3 =  95.24%
 7 =  96.83%
 8 =  93.65%
11 =  98.41%
12 =  96.83%
15 =  95.24%
18 =  98.41%
20 =  98.41%
24 =  96.83%
27 =  84.13%
39 =  92.06%
40 =  98.41%
41 =  93.65%
42 =  95.24%
43 =  93.65%
44 =  98.41%
45 =  93.65%
46 =  95.24%
47 =  96.83%
48 =  95.24%
49 =  93.65%
50 =  95.24%
51 =  95.24%
55 =  92.06%
61 =   3.17%
